In [2]:
import h5py
import matplotlib.pyplot as plt  
import numpy as np

In [3]:
file = "C:\\Users\\SAWINI JANA\\Downloads\\Compressed\\hgcal_electron_data_0001.h5\\hgcal_electron_data_0001.h5"

In [4]:
with h5py.File(file, "r") as f:
    print(f.keys())    

<KeysViewHDF5 ['nhits', 'rechit_energy', 'rechit_x', 'rechit_y', 'rechit_z', 'target']>


In [6]:
with h5py.File(file, "r") as f:
    true_energy = np.array(f["target"])[:50000]

In [77]:
with h5py.File(file, "r") as f:
    
    nhits = np.array(f["nhits"], dtype = int)[:50000]      #fv1            
    rechit_z = f["rechit_z"][: np.sum(nhits)]
    uniq_z = []
    print(uniq_z)
    start = 0

    for i in range(len(nhits)):
        uniq_z.append(np.unique(rechit_z[start: start + nhits[i]]))
        start += nhits[i]
    
    f.close()

[]


In [79]:
z_per_event = []
start = 0

for i in range(len(nhits)):
    z_per_event.append(rechit_z[start: start +nhits[i]])
    start += nhits[i]

print(len(z_per_event))

50000


In [80]:
with h5py.File(file, "r") as f:
    rechit_x = f["rechit_x"][:np.sum(nhits)]
    x_per_event = []
    start = 0

    for i in range(len(nhits)):
        x_per_event.append(rechit_x[start :start+ nhits[i]])
        start += nhits[i]
    print(len(x_per_event))

50000


In [81]:
x_per_event[1]

array([-0.97176361, -0.97176361, -1.94352722, ..., -2.91529083,
        1.94352722,  0.97176361], shape=(1048,))

In [82]:
z_per_event[1]

array([13.87749958, 13.87749958, 13.87749958, ..., 53.9034996 ,
       53.9034996 , 53.9034996 ], shape=(1048,))

In [83]:
with h5py.File(file, "r") as f:
    rechit_y = f["rechit_y"][:np.sum(nhits)]
    y_per_event = []
    start = 0

    for i in range(len(nhits)):
        y_per_event.append(rechit_y[start : start+ nhits[i]])
        start += nhits[i]

    print(len(y_per_event))

50000


In [84]:
print(z_per_event[1])

[13.87749958 13.87749958 13.87749958 ... 53.9034996  53.9034996
 53.9034996 ]


In [85]:
uniq_z[8]


array([13.87749958, 14.76749992, 16.78249931, 17.67250061, 19.6875    ,
       20.57749939, 22.69249916, 23.58250046, 25.69750023, 26.58749962,
       28.70249939, 29.59250069, 31.50749969, 32.39749908, 34.3125    ,
       35.20249939, 37.11750031, 38.00749969, 39.92250061, 40.8125    ,
       42.90750122, 44.03749847, 46.41249847, 47.54249954, 49.68199921,
       50.68799973, 52.88150024])

In [86]:
f = h5py.File(file, "r")
rechit_x = f["rechit_x"][:np.sum(nhits)]
rechit_y = f["rechit_y"][:np.sum(nhits)]

In [87]:
rechit_energy = f["rechit_energy"][:np.sum(nhits)]

In [88]:
rechit_r = np.sqrt(rechit_x**2 + rechit_y **2)

In [ ]:
rechit_r_per_event = []
start = 0
for i in range(len(nhits) ):
    rechit_r_per_event.append(rechit_r[start : start + nhits[i]])
    start += nhits[i]
print(len(rechit_r_per_event))


50000


In [93]:
len(rechit_r_per_event[2])

684

In [95]:
rechit_energy_per_event = []
start = 0

for i in range(len(nhits)):
    rechit_energy_per_event.append(rechit_energy[start:start + nhits[i]])
    start += nhits[i]
    
print(len(rechit_energy_per_event))


50000


In [96]:
len(rechit_energy_per_event[2])

684

In [135]:
dist_z = []
for i in uniq_z:
    z_dist = []
    for idx in range(len(i)-1):
         z_dist.append(i[idx] - i[idx+1])
    dist_z.append(z_dist)

In [ ]:
import numpy as np
from tqdm import tqdm
from tqdm import trange
import csv

rMean_perZ = []
rVar_perZ = []
energy_perZ = []
Etot_perZ = []

for i in trange(50000):

    rMean_perZ_1 = []
    rVar_perZ_1 = []
    energy_perZ_1 = []
    Etot_perZ_l = []

    for k, z_layer in enumerate(uniq_z[i]):
        mask = z_per_event[k] == z_layer
        
        r_hits = rechit_r_per_event[k][mask]

        energy_hits = rechit_energy_per_event[k][mask]

        hit = np.sum(energy_hits)/nhits[i]
        Etot_perZ_l.append(np.sum(energy_hits))
        energy_perZ_1.append(hit)

        if energy_hits.size == 0:
            rMean_perZ_1.append(0.0)
            rVar_perZ_1.append(0.0) 
            continue

        mean = np.average(r_hits)
        var = np.average((r_hits - mean) ** 2)

        rMean_perZ_1.append(mean)
        rVar_perZ_1.append(var)

    rMean_perZ.append(rMean_perZ_1)
    rVar_perZ.append(rVar_perZ_1)
    energy_perZ.append(energy_perZ_1)
    Etot_perZ.append(np.sum(Etot_perZ_l))




100%|██████████| 50000/50000 [00:43<00:00, 1147.47it/s]


In [ ]:
NDF1 = []
import torch
for i in range(len(energy_perZ)):
    NDF1.append(torch.tensor(energy_perZ[i]/Etot_perZ[i]))
    
#contains normalised energy 

In [ ]:
E_diff = []
for i in Etot_perZ:
    Diff_Etot = []
    for idx in range(len(i)-1):
         Diff_Etot.append(i[idx] - i[idx+1])
    E_diff.append(Diff_Etot)

In [129]:
np.sum(rMean_perZ[2])

np.float64(75.94964711178326)

In [99]:
edge_feature  = []
for i in energy_perZ:
    avg_energy = []
    for j in range(len(i)-1):
        avg_energy_perZ = (i[j] + i[j+1]) * 0.5
        avg_energy.append(avg_energy_perZ)
    edge_feature.append(avg_energy)


In [100]:
print(uniq_z[189])

[13.87749958 14.76749992 16.78249931 17.67250061 19.6875     20.57749939
 22.69249916 23.58250046 25.69750023 26.58749962 28.70249939 29.59250069
 31.50749969 32.39749908 34.3125     35.20249939 37.11750031 38.00749969
 39.92250061 40.8125     42.90750122 44.03749847 46.41249847 49.68199921
 50.68799973 52.88150024 53.9034996 ]


In [101]:
edge_index = [] #for each event that is 50000, there will be edge index

from tqdm import trange
import torch

for i in trange(50000, desc="HI"):
    # Create edge index: i → i+1
    num_nodes = len(uniq_z[i])
    source_nodes = np.arange(num_nodes - 1)     
    target_nodes = np.arange(1, num_nodes)     

    # Stack into edge_index
    edge_index.append(np.stack([source_nodes, target_nodes], axis=0))
    


HI: 100%|██████████| 50000/50000 [00:00<00:00, 168462.99it/s]


In [102]:
len(edge_index[189][0]), len(uniq_z[189])

(26, 27)

In [103]:
with h5py.File(file, "r") as f:
        rechit_energy = f["rechit_energy"][:np.sum(nhits)] 
        rechit_energy_per_event = []
        start = 0
        for i in range(len(nhits)):
                rechit_energy_per_event.append(rechit_energy[start:start + nhits[i]])
                start += nhits[i] 

In [104]:
print(len(rechit_energy_per_event))

50000


In [113]:
def E1_E7_E19(x_list, y_list, max_x, max_y, energy, max_energy):
    dist_x = np.zeros(len(x_list))
    dist_x = np.full_like(dist_x, max_x)

    dist_del_x = np.square(dist_x - x_list)
    
    dist_y = np.zeros(len(y_list))
    dist_y = np.full_like(dist_y, max_y)
    dist_del_y = np.square(dist_y - y_list)
    
    distance = np.sqrt(dist_del_x + dist_del_y)
    #print(f"Length of distance: {len(distance)} | Length of dist_x: {len(dist_x)}")
    if len(distance) > 5:
        
        dist_sorted_7 = np.sort(np.unique(distance))[1]
        #print(dist_sorted_7)
        if np.all(dist_sorted_7 < 1): 
            
            indices_7 = np.where(dist_sorted_7 == distance)
            energy_hits = energy[indices_7]
            E1_E7_ratio = (np.sum(energy_hits) + max_energy)/ max_energy
        else:
            E1_E7_ratio = 1
    else:
        E1_E7_ratio = 1


    if len(distance) > 17:
        
        dist_sorted_19 = np.sort(np.unique(distance))[1:4]
        #print(dist_sorted_19)
        if np.all(dist_sorted_19 < 3):
            
            indices_19 = np.where(np.isin(distance, dist_sorted_19))[0]
            energy_hits = energy[indices_19]
            E7_E19_ratio = (np.sum(energy_hits) + max_energy)/ max_energy
        else:
            E7_E19_ratio = 1
    else:
        E7_E19_ratio = 1


    return E1_E7_ratio, E7_E19_ratio 



    



In [114]:
E1_E7 = []
E7_E19 = []

for i in trange(len(rechit_energy_per_event), desc="HI"):
    # i denotes the event
    #going into each event
    z = np.unique(uniq_z[i])  #returns all the z-cordinates / layers in that event
    E1_E7_l = []
    E7_E19_l = []
    #print(f"for: {i} event")

    for j in z:
        #print(f"Layer: {j}")
        mask = np.where(z_per_event[i] == j)[0]
        energy = rechit_energy_per_event[i][mask] #returns all the energy hits in that layer
        
        max_energy = np.max(energy) # for each event for each layer I have the max energy
        max_index = np.where(max_energy == rechit_energy_per_event[i])[0] #co-ordinate of that hit
        if len(max_index) > 1:
            max_index = max_index[0]
         
        #print(rechit_energy_per_event[i][max_index])
        x_hit = x_per_event[i][max_index] 
        y_hit = y_per_event[i][max_index] 

        x_indices = x_per_event[i][mask] #returns all the x-co-ordinates in that layer
        y_indices = y_per_event[i][mask] #returns all the y co-ordinates in that layer
        
        ND1 , ND2 = E1_E7_E19(x_list= x_indices,
                              y_list= y_indices,
                              max_x= x_hit,
                              max_y=y_hit,
                              max_energy= max_energy,
                              energy= energy)

        E1_E7_l.append(ND1)
        E7_E19_l.append(ND2)
    
    E1_E7.append(E1_E7_l)
    E7_E19.append(E7_E19_l)
    


HI: 100%|██████████| 50000/50000 [01:14<00:00, 670.26it/s]


In [115]:
print(len(E1_E7))

50000


In [48]:
x_per_event[1]

array([], dtype=float64)

In [49]:
y_per_event[1]

array([], dtype=float64)

In [ ]:
[0.         1.25909904 1.25909914 1.25909927 1.2590993  1.25909934 1.25909941]

In [306]:
(1.25909914 - 1.25909904)/1.25909914 * 100
(1.25909927 - 1.25909914) / 1.25909927

1.0324841193665943e-07

0.1755600025664038

In [317]:
(0.16229211 - 0.07527062 ) / 0.16229211
0.17134148 * 1.15

0.197042702

In [316]:
(0.75053796 - 0.33952787) /0.75053796
0.92952382 * 1.15

1.068952393

In [388]:
#I have the nodes = uniq_z [50,000 X number of layers... ]
#I have the edge_indices 

In [116]:
#I have the node features: E1_E7 , E7_E19, rMean, rSTd, nhits [50,000 X number of layers]
len(rMean_perZ[102]), len(rVar_perZ[102]), len(E1_E7[102]), len(E7_E19[102]), nhits[102]

(28, 28, 28, 28, np.int64(1208))

In [ ]:
#I have the edge features as rechit_energy per layer averaged over nhits 

In [117]:
#stack all the node features together
hits = [nhits[102]] * len(uniq_z[102])
print(hits)
a = torch.stack((torch.tensor(rMean_perZ[102]), torch.tensor(rVar_perZ[102]), torch.tensor(E1_E7[102]), torch.tensor(E7_E19[102]), torch.tensor(hits)), dim=1)
print(a.shape)

[np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208), np.int64(1208)]
torch.Size([28, 5])


In [118]:
for i in range(50000):
    if len(torch.tensor(edge_index[i][0])) != len(torch.tensor(edge_feature[i])):
        print(i)
    

In [132]:
print(len(E1_E7[0]))

28


In [134]:
E1_E7_ratio = []
for k in range(50000):
    E1_E7_ratio_l = []
    for i in range(len(E1_E7[k]) - 1):
        E1_E7_ratio_l.append((E1_E7[k][i] + E1_E7[k][i+1])/2)
    E1_E7_ratio.append(E1_E7_ratio_l)

E7_E19_ratio = []
for k in range(50000):
    E7_E19_ratio_l = []
    for i in range(len(E7_E19[k]) - 1):
        E7_E19_ratio_l.append((E7_E19[k][i] + E7_E19[k][i+1])/2)
    E7_E19_ratio.append(E7_E19_ratio_l)
     


In [ ]:
from torch_geometric.data import Data
import torch

data_list = []

for i in trange(50000, desc="PROCESSING"):
    #preparing node features
    hits = [nhits[i]] * len(uniq_z[i])
    x = torch.stack((torch.tensor(rMean_perZ[i], dtype=torch.float),
                    torch.tensor(rVar_perZ[i], dtype=torch.float),
                    torch.tensor(energy_perZ[i], dtype = torch.float),
                    torch.tensor(NDF1[i], dtype = torch.float),
                    torch.tensor(E1_E7[i], dtype=torch.float), 
                    torch.tensor(E7_E19[i], dtype=torch.float),
                    ), dim=1)

    #preparing edge_index
    edge = torch.tensor(edge_index[i], dtype = torch.long)

    #preparing edge_feature
    edge_attr = torch.stack((
                            torch.tensor(E_diff[i], dtype = torch.float),
                            torch.tensor(dist_z[i], dtype = torch.float)
                            ), dim = 1)

    #preparing data label
    label = torch.tensor(true_energy[i], dtype=torch.float)

    data = Data(
        x = x,
        edge_index= edge,
        edge_attr= edge_attr,
        y = label
    )

    data_list.append(data)

torch.save(data_list,'D:\\my_graph_data_electron.pt')


PROCESSING: 100%|██████████| 50000/50000 [00:05<00:00, 9172.11it/s] 


In [125]:
dataset = torch.load("D:\\my_graph_data_electron.pt")
cleaned_dataset = [g for g in dataset if g.x.shape[0] > 0]
torch.save(cleaned_dataset, "D:\\cleaned_graph_data_electron.pt")

C:\Users\SAWINI JANA\AppData\Local\Temp\ipykernel_3152\4008397149.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load("D:\\my_graph_data_electron.pt")


In [126]:
for i in dataset:
    print(i.x.shape)
    c = 0
    if c == 10:
        break
        c += 1



torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([27, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([27, 4])
torch.Size([20, 4])
torch.Size([26, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([27, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([23, 4])
torch.Size([27, 4])
torch.Size([28, 4])
torch.Size([24, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([26, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([22, 4])
torch.Size([27, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([27, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])
torch.Size([28, 4])


In [127]:
dataset_1 = torch.load("D:\\cleaned_graph_data_electron.pt")

C:\Users\SAWINI JANA\AppData\Local\Temp\ipykernel_3152\4089484892.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset_1 = torch.load("D:\\cleaned_graph_data_electron

In [128]:
len(dataset_1)

50000